1) Metric - ndcg, recall

2) Loss Function

In [1]:
try:
    %tensorflow_version 2.x
except:
    pass

import tensorflow as tf
tf.__version__

TensorFlow 2.x selected.


'2.0.0'

In [0]:
import os
from os.path import basename, normpath
import urllib.request
import requests
import tarfile
import tempfile
import zipfile
from tqdm import tqdm
import itertools

from glob import glob

In [0]:
import scipy.sparse as sp
import pandas as pd
import numpy as np

In [0]:
def argsort_sparse(m_sp, R=None):
    '''

    Parameters
    ----------
    m_sp : scipy sparse matrix

    R(optional) : int
        maximum number of keeping indexs

    Returns
    -------
    Sorted Indexs for nonzero data
    '''
    row_inds, col_inds = m_sp.nonzero()
    tuples = zip(row_inds, col_inds, m_sp.data)
    tuples = sorted(tuples, key=lambda tup: tup[1])
    tuples = sorted(tuples, key=lambda tup: tup[2], reverse=True)
    sorted_tuples = sorted(tuples, key=lambda tup: tup[0])
    #sorted_tuples = list(sorted(tuples)) # priority --> data < col_ins < row_inds

    n_rows, n_cols = m_sp.shape

    results = []
    tup_idx = 0
    for r in range(n_rows):
        results.append([])
        for i, tup in enumerate(sorted_tuples[tup_idx:]):
            if tup[0] == r:
                results[-1].append(tup[1])
            else:
                tup_idx += i
                break
        
        temp = 0
        while len(results[-1]) < R and temp < n_cols:
            if not temp in results[-1]:
                results[-1].append(temp)

            temp += 1

    if R is not None:
        results = list(map(lambda l:l[:R], results))



    return np.array(results)


    

def recall_at_r(x_true, x_predicted, R):
    '''
    
    Parameters
    ----------
    x_true : scipy sparse matrix or numpy array
        true data
    x_predicted : numpy array
        predicted recommendations
    R : int 
        hyper-parameter for this metric

    Returs
    ------
    Recall@R

    '''

    if type(x_true) == np.array:
        sorted_col_inds = np.argsort(x_true, axis=-1).reshape(-1)
    else:
        sorted_col_inds = argsort_sparse(x_true, R).reshape(-1)



    row_inds = np.repeat(np.array(list(range(x_true.shape[0]))), R)


    x_true_cp = sp.csr_matrix(([1]*(R*x_true.shape[0]), (list(row_inds), list(sorted_col_inds)) ), shape=x_true.shape)
    sorted_idxs_predicted = np.argsort(x_predicted, axis=-1)
    selected = np.take_along_axis(x_true_cp, sorted_idxs_predicted[:, :R], axis=-1)
    hit = selected.sum(axis=-1)
    maxhit = np.minimum(x_true.getnnz(axis=1), R)

    return np.squeeze(np.array(hit)) / maxhit

def ndcg(x_true, x_predicted, R):
    '''

    Parameters
    ----------
    x_true : numpy array or scipy sparse matrix
      
    x_predicted : numpy array or tensorflow tensor

    R : int


    Returns
    -------
    Normalized Discounted Cumulative Gain(ndcg)


    '''

    #TODO : if there is not enough interaction between users and items, or the minimum items that users interacte with is less than R, we need to fix it

    n_rows, n_cols = x_true.shape

    if type(x_true) == np.array:
        sorted_col_inds = np.argsort(x_true, axis=-1).reshape(-1)
    else:
        sorted_col_inds = argsort_sparse(x_true, R).reshape(-1)



    row_inds = np.repeat(np.array(list(range(x_true.shape[0]))), R)

    #x_true_cp = if there is interation and within R --> 1. otherwise, 0
    x_true_cp = sp.csr_matrix(([1]*(R*x_true.shape[0]), (list(row_inds), list(sorted_col_inds)) ), shape=x_true.shape)
    sorted_idxs_predicted = np.argsort(x_predicted, axis=-1)[:, ::-1]
    '''
    results = np.zeros((n_rows, 1))
    for r in range(1, R+1, 1):
        selected = np.take_along_axis(x_true_cp, sorted_idxs_predicted[:, r-1:r], axis=-1).toarray()
        denominator = np.log(r+1)
        nominator = np.power(2, selected) - 1
        results += nominator / denominator
    '''

    selected = np.take_along_axis(x_true_cp, sorted_idxs_predicted[:,:R], axis=-1).toarray() #(batch, R)
    #numerator = np.power(2, selected) - 1.
    denominator = np.expand_dims(np.log2(np.arange(2,R+2,1)), axis=0)
    results = selected / denominator
    results = results.sum(axis=-1) #(batch)
    #normalizer = np.expand_dims(np.power(2, np.array([1 for _ in range(R)])), axis=0) - 1.
    normalizer = np.expand_dims(np.array([1 for _ in range(R)]), axis=0)
    normalizer = normalizer / denominator
    normalizer = normalizer.sum(axis=-1)

    return results / normalizer #(batch)

In [5]:
sample_true = sp.csr_matrix((list(range(20)), (np.repeat(np.array(list(range(4))), 5),np.tile([1,2,3,4,5], 4))), shape=(4,10))
sample_true.toarray()

array([[ 0,  0,  1,  2,  3,  4,  0,  0,  0,  0],
       [ 0,  5,  6,  7,  8,  9,  0,  0,  0,  0],
       [ 0, 10, 11, 12, 13, 14,  0,  0,  0,  0],
       [ 0, 15, 16, 17, 18, 19,  0,  0,  0,  0]], dtype=int64)

In [6]:
sample_predicted = sp.csr_matrix((list(range(20)), (np.repeat(np.array(list(range(4))), 5),np.tile([1,2,3,4,5], 4))), shape=(4,10))
sample_predicted = sample_predicted.toarray()
sample_predicted

array([[ 0,  0,  1,  2,  3,  4,  0,  0,  0,  0],
       [ 0,  5,  6,  7,  8,  9,  0,  0,  0,  0],
       [ 0, 10, 11, 12, 13, 14,  0,  0,  0,  0],
       [ 0, 15, 16, 17, 18, 19,  0,  0,  0,  0]], dtype=int64)

In [7]:
ndcg(sample_true, sample_predicted, 3)

array([1., 1., 1., 1.])

In [0]:
DATASETS_DIR = './'
# the different datasets
ML_20M = 'ml-20m'
ML_20M_ALT = 'ml-20m_alt'
NETFLIX = 'netflix'
LASTFM = 'lastfm'
PINTEREST = 'pinterest'
DATASETS = [ML_20M, NETFLIX, LASTFM, PINTEREST, ML_20M_ALT]

# download urls to different datasets
DOWNLOAD_URL = {
    ML_20M: 'http://files.grouplens.org/datasets/movielens/ml-20m.zip',
    NETFLIX: 'https://archive.org/download/nf_prize_dataset.tar/nf_prize_dataset.tar.gz',
    LASTFM: 'http://mtg.upf.edu/static/datasets/last.fm/lastfm-dataset-360K.tar.gz',
}

In [0]:
def download_file(url, filename):
    if not os.path.isdir(DATASETS_DIR):
        os.makedirs(DATASETS_DIR)

    u = urllib.request.urlopen(url)
    with open(filename, 'wb') as f:
        meta = u.info()
        if (meta.get_all("Content-Length")):
            file_size = int(meta.get_all("Content-Length")[0])
            pbar = tqdm(
                total=file_size,
                desc=basename(normpath(filename)),
                unit='B',
                unit_scale=True)

            file_size_dl = 0
            block_sz = 8192
            while True:
                buff = u.read(block_sz)
                if not buff:
                    break
                pbar.update(len(buff))
                file_size_dl += len(buff)
                f.write(buff)
            pbar.close()
        else:
            LOG.warning("No content length information")
            file_size_dl = 0
            block_sz = 8192
            for cyc in itertools.cycle('/–\\|'):
                buff = u.read(block_sz)
                if not buff:
                    break
                print(cyc, end='\r')
                file_size_dl += len(buff)
                f.write(buff)



def extract_file(path, to_directory):
    """
    Extract file
    :param path: Path to compressed file
    :param to_directory: Directory that is going to store extracte files
    """
    if (path.endswith("tar.gz")):
        tar = tarfile.open(path, "r:gz")
        tar.extractall(path=to_directory)
        tar.close()
    elif (path.endswith("tar")):
        tar = tarfile.open(path, "r:")
        tar.extractall(path=to_directory)
        tar.close()
    elif (path.endswith("zip")):
        with zipfile.ZipFile(path, 'r') as zip_ref:
            zip_ref.extractall(to_directory)
    else:
        raise Exception(
            "Could not extract {} as no appropriate extractor is found".format(path))

def download_movielens():
    filepath = os.path.join(DATASETS_DIR, ML_20M_ALT + '.zip')
    if not glob(filepath):
        download_file(DOWNLOAD_URL[ML_20M], filepath)

    #.info("Extracting")
    extract_file(filepath, DATASETS_DIR)

def download_lastfm():
    filepath = os.path.join(DATASETS_DIR, LASTFM + '.tar.gz')
    if not glob(filepath):
        download_file(DOWNLOAD_URL[LASTFM], filepath)

    extract_file(filepath, DATASETS_DIR)


In [11]:
download_movielens()

ml-20m_alt.zip: 100%|██████████| 199M/199M [00:07<00:00, 26.0MB/s]


In [0]:
def make_feedback_implicit(feedback, threshold):
    return [1 if rating >= threshold else 0 for rating in feedback]

def parse_movielens(threshold=4, **kwargs):

    source_file = './ml-20m/ratings.csv'
    if not glob(source_file):
        download_movielens()


    df = pd.read_csv(source_file)
    df.drop('timestamp', axis=1, inplace=True)
    df['rating'].fillna(0.)
    df['rating'] = make_feedback_implicit(df['rating'], 3.5)

    map_user_id = {u: i for i, u in enumerate(df.userId.unique())}
    map_movie_id = {m: i for i, m in enumerate(df.movieId.unique())}

    m_sp = sp.csr_matrix(
        (df.rating,
         ([map_user_id[u] for u in df.userId],
          [map_movie_id[m] for m in df.movieId])),
        shape=(len(map_user_id), len(map_movie_id))
    )

    m_sp.eliminate_zeros()

    def save_as_npz(m_sp, path):
        if not os.path.isdir('./binary'):
            os.mkdir('./binary')
        sp.save_npz(path, m_sp)
    save_as_npz(m_sp, './binary/bin_ml-20m.npz')

In [0]:
def parse_lastfm():
    df = pd.read_csv('./lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv', delimiter='\t', names=['UserId', 'MusicId', 'ArtistName','Plays'])

    music_artist_pair = list(zip([str(i) for i in df['MusicId']],[str(i) for i in df['ArtistName']]))

    user_id_dict = {id: i for i, id in enumerate(sorted(set(df['UserId'])))}
    item_id_dict = {key: i for i, key in enumerate(sorted(set(music_artist_pair)))}

    user_idxs = [user_id_dict[user] for user in df['UserId']]
    item_idxs = [item_id_dict[item] for item in music_artist_pair]

    m_sp = sp.csr_matrix(([1] * df.shape[0], (user_idxs, item_idxs)), shape=(len(user_id_dict), len(item_id_dict)))

    def save_as_npz(m_sp, path):
        if not os.path.isdir('./binary'):
            os.mkdir('./binary')
        sp.save_npz(path, m_sp)

    save_as_npz(m_sp, './binary/lastfm.npz')
    del user_idxs
    del item_idxs
    del music_artist_pair
    del df


In [0]:
parse_movielens()

In [0]:
data = sp.load_npz('./binary/bin_ml-20m.npz')

In [0]:
num_users, num_items = data.shape


In [10]:
num_users // 500

276

In [11]:
data.getnnz()

12195566

In [0]:
from sklearn.model_selection import train_test_split
train_x, val_x = train_test_split(data, test_size=0.2)

In [0]:
def make_generator(sparse_matrix, batch_size):
    n, _ = sparse_matrix.shape
    buckets = n // batch_size
    additional = (n % batch_size != 0)
    def generator():
        while True:
            i = 0
            while i < buckets:
                batch = sparse_matrix[i*batch_size:(i+1) * batch_size].copy()
                batch = batch.tocoo()
                idxs = np.stack([batch.row, batch.col], axis=1)
                vals = batch.data
                yield (idxs, vals)
                i += 1

            if additional:
                batch = sparse_matrix[i*batch_size:]
                batch = batch.tocoo()
                idxs = np.stack([batch.row, batch.col], axis =1)
                vals = batch.data
                yield (idxs, vals)

    return generator

In [0]:
BATCH_SIZE = 500

In [0]:
gen = make_generator(train_x, BATCH_SIZE)
_, movies = data.shape

In [0]:
dataset_v1 = tf.data.Dataset.from_generator(gen, output_types=(tf.int64, tf.float32)).map(lambda i, v: tf.sparse.SparseTensor(i, v, (BATCH_SIZE, movies)))

In [0]:
def make_generator_v2(sparse_matrix, batch_size):
    n, _ = sparse_matrix.shape
    buckets = n // batch_size
    additional = (n % batch_size != 0)
    def generator():
        while True:
            i = 0
            while i < buckets:
                batch = sparse_matrix[i*batch_size:(i+1) * batch_size].copy()
                yield batch.toarray()
                i += 1

            if additional:
                batch = sparse_matrix[i*batch_size:]
                yield batch.toarray()

    return generator

In [0]:
gen_v2 = make_generator_v2(train_x, BATCH_SIZE)

In [0]:
dataset_v2 = tf.data.Dataset.from_generator(gen_v2, output_types=tf.float32)

In [0]:
def make_val_generator(sparse_matrix, batch_size):
    n, _ = sparse_matrix.shape
    buckets = n // batch_size
    additional = (n % batch_size != 0)
    def generator():
        i = 0
        while i < buckets:
            batch = sparse_matrix[i*batch_size:(i+1) * batch_size].copy()
            yield batch.toarray()
            i += 1

        if additional:
            batch = sparse_matrix[i*batch_size:]
            yield batch.toarray()

    return generator

In [0]:
gen_val = make_val_generator(val_x, BATCH_SIZE)

In [0]:
val_dataset = tf.data.Dataset.from_generator(gen_val, output_types=tf.float32)

Ratings ==> User * Movies

Movie_id ==> Movie Titile

In [0]:
class Sparse2Dense(tf.keras.layers.Dense):
    def call(self, x):
        #print(type(x))
        assert type(x) == tf.sparse.SparseTensor
        rank = len(x.shape)
        if rank != 2:
            raise NotImplementedError("input rank should be 2")
        else:
            outputs = tf.sparse.sparse_dense_matmul(x, self.kernel)
        if self.use_bias:
            outputs = tf.nn.bias_add(outputs, self.bias)
        if self.activation is not None:
            return self.activation(outputs)  # pylint: disable=not-callable
        return outputs

In [0]:
class VAE_CF_sparse_v1(tf.keras.Model):
    def __init__(self, items, hidden_dims, latent_dims, *args, **kwargs):
        super(VAE_CF_sparse_v1, self).__init__(*args, **kwargs)
        self.encoder = tf.keras.Sequential([
                #tf.keras.layers.Dense(hidden_dims),
                Sparse2Dense(hidden_dims),
                tf.keras.layers.Dense(hidden_dims),
                tf.keras.layers.Dense(2 * latent_dims)
        ])

        self.decoder = tf.keras.Sequential([
                #tf.keras.layers.InputLayer(input_shape=(latent_dims)),
                tf.keras.layers.Dense(hidden_dims),
                tf.keras.layers.Dense(hidden_dims),
                tf.keras.layers.Dense(items)
        ])

    def encode(self, x):
        mean, log_var = tf.split(self.encoder(x), 2, 1)
        z = self.reparameterize(mean, log_var)
        return mean, log_var, z

    def decode(self, z):
        return self.decoder(z)

    def reparameterize(self, mean, log_var):
        batch = tf.shape(mean)[0]
        dim = tf.shape(mean)[1]
        epsilon = tf.random.normal(shape=(batch, dim))
        return mean + tf.exp(log_var * .5) * epsilon

    def call(self, inputs):
        mean, log_var, z = self.encode(inputs)
        reconstructed = self.decode(z)
        kl_loss = -.5 * tf.math.reduce_mean((log_var - tf.exp(log_var) - tf.square(mean) + 1))
        self.add_loss(kl_loss)
        return reconstructed


In [0]:
def encoder_layer(items, hidden_dims, latent_dims, batch_size):
    inputs = tf.keras.layers.Input(shape=(items,), sparse=True, batch_size=batch_size)
    x = Sparse2Dense(hidden_dims)(inputs)
    x = tf.keras.layers.Dense(hidden_dims)(x)
    outputs = tf.keras.layers.Dense(2 * latent_dims)(x)

    return tf.keras.Model(inputs=inputs, outputs=outputs)

def decoder_layer(items, hidden_dims, latent_dims):
    inputs = tf.keras.layers.Input(shape=(latent_dims,), batch_size=BATCH_SIZE)
    x = tf.keras.layers.Dense(hidden_dims)(inputs)
    x = tf.keras.layers.Dense(hidden_dims)(x)
    outputs = tf.keras.layers.Dense(items)(x)

    return tf.keras.Model(inputs=inputs, outputs=outputs)

In [0]:
class VAE_CF_sparse_v2(tf.keras.Model):
    def __init__(self, items, hidden_dims, latent_dims, *args, **kwargs):
        super(VAE_CF_sparse_v2, self).__init__(*args, **kwargs)
        self.encoder = encoder_layer(items, hidden_dims, latent_dims, BATCH_SIZE)

        self.decoder = decoder_layer(items, hidden_dims, latent_dims)

    def encode(self, x):
        mean, log_var = tf.split(self.encoder(x), 2, 1)
        z = self.reparameterize(mean, log_var)
        return mean, log_var, z

    def decode(self, z):
        return self.decoder(z)

    def reparameterize(self, mean, log_var):
        batch = tf.shape(mean)[0]
        dim = tf.shape(mean)[1]
        epsilon = tf.random.normal(shape=(batch, dim))
        return mean + tf.exp(log_var * .5) * epsilon

    def call(self, inputs):
        mean, log_var, z = self.encode(inputs)
        reconstructed = self.decode(z)
        #kl_loss = -.5 * tf.math.reduce_mean((log_var - tf.exp(log_var) - tf.square(mean) + 1))
        #print("kl loss", kl_loss)
        #self.add_loss(kl_loss)
        return reconstructed

In [0]:
class VAE_CF_dense_v1(tf.keras.Model):
    def __init__(self, items, hidden_dims, latent_dims, *args, anneal_steps_total=20000, anneal_cap=0.2, **kwargs):
        super(VAE_CF_dense_v1, self).__init__(*args, **kwargs)

        self.beta = 0.0
        self.anneal_cap = anneal_cap
        self.anneal_steps_total = anneal_steps_total

        self.dropout = tf.keras.layers.Dropout(0.5)
        self.encoder = tf.keras.Sequential([
                #tf.keras.layers.InputLayer(input_shape=(items,)),
                tf.keras.layers.Dense(hidden_dims, name='encoder_dense_1'),
                tf.keras.layers.Dense(hidden_dims, name='encoder_dense_2'),
                tf.keras.layers.Dense(2 * latent_dims, name='encoder_dense_final')
        ])

        self.decoder = tf.keras.Sequential([
                #tf.keras.layers.InputLayer(input_shape=(latent_dims, )),
                tf.keras.layers.Dense(hidden_dims, name='decoder_dense_1'),
                tf.keras.layers.Dense(hidden_dims, name='decoder_dense_2'),
                tf.keras.layers.Dense(items, name='decoder_dense_final'),
                tf.keras.layers.LayerNormalization()
        ])

    def anneal_step(self, update_step):
        self.beta = min(self.anneal_cap, self.anneal_cap * update_step / self.anneal_steps_total)

    def encode(self, x, training=True):
        inp = self.dropout(x, training)
        mean, log_var = tf.split(self.encoder(inp), 2, 1)
        z = self.reparameterize(mean, log_var)
        return mean, log_var, z

    def decode(self, z):
        return self.decoder(z)

    def reparameterize(self, mean, log_var):
        batch = tf.shape(mean)[0]
        dim = tf.shape(mean)[1]
        epsilon = tf.random.normal(shape=(batch, dim))
        return mean + tf.exp(log_var * .5) * epsilon

    def call(self, inputs, training=True):
        mean, log_var, z = self.encode(inputs, training)
        reconstructed = self.decode(z)
        kl_loss = -.5 * tf.math.reduce_mean((log_var - tf.exp(log_var) - tf.square(mean) + 1))
        #print(self.beta*kl_loss)
        self.add_loss(self.beta*kl_loss)
        return reconstructed

In [43]:
'''
explicit
mse
poisson

implicit
weighted mse
multinomial
sigmoid_cross_entropy
'''

'\nexplicit\nmse\npoisson\n\nimplicit\nweighted mse\nmultinomial\nsigmoid_cross_entropy\n'

In [0]:
def get_loss_fn(loss_fn_name='multinomial'):
    assert loss_fn_name.lower() in {'mse', 'poisson', 'multinomial', 'sigmoid_ce', 'weighted_mse'}
    if loss_fn_name == 'mse':
        return tf.keras.losses.MeanSquaredError()
    elif loss_fn_name == 'poisson':
        raise NotImplementedError("poisson")
    elif loss_fn_name == 'multinomial':
        #return tf.keras.losses.CategoricalCrossentropy(from_logits=True)
        def multinomial(labels, logits):
            return -tf.math.reduce_mean(tf.math.reduce_sum(tf.nn.log_softmax(logits, axis=1) * labels, axis=-1))
        
        return multinomial
    elif loss_fn_name == 'sigmoid_ce':
        return lambda label, logit: tf.nn.sigmoid_cross_entropy_with_logits(label, logit)
    else:
        raise NotImplementedError("weighted_mse")
        

In [0]:
loss_fn = get_loss_fn()

In [0]:
vae_cf = VAE_CF_dense_v1(num_items, 400, 200)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3, clipnorm=True)
loss_fn = get_loss_fn()
loss_metric = tf.keras.metrics.Mean()

In [0]:
idx = 0
for i in iter(dataset_v2):
    if idx < 2:
        vae_cf(i)
    else:
        break
    idx += 1

In [48]:
vae_cf.losses

[<tf.Tensor: id=456, shape=(), dtype=float32, numpy=0.0>]

In [0]:
@tf.function
def train_step(model, x, loss_fn, optimizer):
    #if type(x) == tf.sparse.SparseTensor:
    #    x = tf.sparse.to_dense(x)
    with tf.GradientTape() as tape:
        logit = model(x, training=True)
        if type(x) == tf.sparse.SparseTensor:
            x = tf.sparse.to_dense(x)
        loss = loss_fn(x, logit)
        loss += sum(model.losses)



    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return loss

In [0]:
#@tf.function
def val_loss(model, val_dataset, loss_fn):
    assert isinstance(val_dataset, tf.data.Dataset)

    loss_metric = tf.keras.metrics.Mean()
    for val_x in val_dataset:
        logit = model(val_x, training=False)
        if type(x) == tf.sparse.SparseTensor:
            val_x = tf.sparse.to_dense(val_x)
        loss = loss_fn(val_x, logit)
        loss += sum(model.losses)
        loss_metric(loss)
    return loss_metric.result()

In [67]:
EPOCHS = 1
STEPS_PER_EPOCH = num_users // BATCH_SIZE
update_step = 0
train_loss_track = []
val_loss_track = []

for epoch in range(EPOCHS):

    for step, x in enumerate(dataset_v2):
        #print(type(x) == tf.sparse.SparseTensor)
        vae_cf.anneal_step(update_step)
        loss = train_step(vae_cf, x, loss_fn, optimizer)
        loss_metric(loss)

        update_step += 1

        if step % 20 == 0:
            print("epoch {} at step {} loss = {}".format(epoch, step, loss))
            train_loss_track.append(loss.numpy())
            val_loss_track.append(val_loss(vae_cf, val_dataset, loss_fn).numpy())

        if step == STEPS_PER_EPOCH:
            break 

epoch 0 at step 0 loss = 658.8325805664062
epoch 0 at step 20 loss = 722.103271484375
epoch 0 at step 40 loss = 640.103515625
epoch 0 at step 60 loss = 673.4186401367188
epoch 0 at step 80 loss = 648.980712890625
epoch 0 at step 100 loss = 719.493408203125
epoch 0 at step 120 loss = 733.8172607421875
epoch 0 at step 140 loss = 676.2261352539062
epoch 0 at step 160 loss = 605.3428955078125
epoch 0 at step 180 loss = 687.110595703125
epoch 0 at step 200 loss = 702.1478271484375
epoch 0 at step 220 loss = 655.8302612304688
epoch 0 at step 240 loss = 640.4595336914062
epoch 0 at step 260 loss = 608.4702758789062


In [68]:
val_loss_track

[699.2807,
 698.0057,
 696.8747,
 4.601901e+25,
 83480.55,
 2875.6814,
 3851.439,
 663.2912,
 661.61053,
 inf,
 20374.018,
 36171.938,
 861.29443,
 672.587]

In [0]:
'''
validation
held_out user(validation set)
held_out item(some itmes that does not participate in reconstruction)

0) held_out user should interact with at least K items(filter out) o
1) randomly select zero-out items and store indices
2) construct new gt interaction matrix at stored indices above one is populated
3) input --> zero_out, Dropout off

'''